In [16]:
# !pip install yfinance

In [17]:
# !pip install pandas_ta

In [18]:
# !pip install hvplot

In [19]:
# !pip install ta

In [20]:
# Import the required libraries
import numpy as np
import pandas as pd
import yfinance as yf
import ta
import pandas_ta as ta
import hvplot.pandas
from ta.trend import SMAIndicator, macd, PSARIndicator, MACD
from ta.volatility import BollingerBands
from ta.momentum import rsi, StochasticOscillator
from ta import add_all_ta_features, add_trend_ta, add_volume_ta, add_volatility_ta, add_momentum_ta, add_others_ta
from pandas_datareader import data as pdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from datetime import datetime
import math

In [32]:
# BTC Price Data
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)
    
    
atr_period = 6
atr_multiplier = 3.0

symbol = 'BTC-USD'
df = yf.download(symbol, start='2020-01-01')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)
def backtest_supertrend(df, investment):
    is_uptrend = df['Supertrend']
    close = df['Close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = (equity / close[i] / 100) * 100
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} BTC at {round(close[i],)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell BTC at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100k in Bitcoin is ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity
entry, exit, roi = backtest_supertrend(df, 100000)

# df.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close']),
                go.Scatter(x=df.index, y=df['Final Upperband'], line=dict(color='red', width=1), name="Final Upperband"),
                go.Scatter(x=df.index, y=df["Final Lowerband"], line=dict(color='green', width=1), name="Final Lowerband")])         
                

fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(title="Bitcoin Price Data 2020 to Present", yaxis_title="Price in USD", autosize=False, width=1600, height=800,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
# update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
fig.update_layout(height=900, width=1600, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)])

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)

# Plot volume trace on 2nd row
colors = ['green' if row['Open'] - row['Close'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors
                    ), row=2, col=1)

# Assigning MACD
btc_df = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            )
# Plot MACD trace on 3rd row
fig.add_trace(go.Bar(x=df.index, 
                     y=btc_df.macd_diff()
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=btc_df.macd(),
                         line=dict(color='black', width=2)
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=btc_df.macd_signal(),
                         line=dict(color='blue', width=1)
                        ), row=3, col=1)
# Plot MACD trace on 3rd row
colors = ['green' if val >= 0 
          else 'red' for val in btc_df.macd_diff()]
fig.add_trace(go.Bar(x=df.index, 
                     y=btc_df.macd_diff(),
                     marker_color=colors
                    ), row=3, col=1)


[*********************100%***********************]  1 of 1 completed
Buy 13.614918387273029 BTC at 7345 on 2020/01/03
Sell BTC at 9341.71 on 2020/02/25
Buy 17.48973846492507 BTC at 7272 on 2020/04/06
Sell BTC at 8601.8 on 2020/05/11
Buy 14.796320150605178 BTC at 10167 on 2020/06/01
Sell BTC at 10245.3 on 2020/09/03
Buy 13.419161108372199 BTC at 11296 on 2020/10/10
Sell BTC at 30432.55 on 2021/01/27
Buy 8.839945888391146 BTC at 46196 on 2021/02/08
Sell BTC at 47093.85 on 2021/02/25
Buy 7.201820385658462 BTC at 57805 on 2021/03/11
Sell BTC at 55724.27 on 2021/04/19
Buy 11.352447625743771 BTC at 35350 on 2021/07/25
Sell BTC at 46091.39 on 2021/09/08
Buy 10.653923370395091 BTC at 49113 on 2021/10/04
Sell BTC at 56942.14 on 2021/11/18
Buy 14.617816158047821 BTC at 41501 on 2022/02/04
Sell BTC at 38431.38 on 2022/02/20
Buy 13.006153386767288 BTC at 43193 on 2022/02/28
Sell BTC at 43206.74 on 2022/04/06
Buy 24.991382479034883 BTC at 22486 on 2022/07/18
Earning from investing $100k in Bitcoin 

In [22]:
# BTC Optimal Parameters
def find_optimal_parameter(df):
    # predefine several parameter sets
    atr_period = [6, 7, 8, 9, 10]
    atr_multiplier = [1.0, 1.5, 2.0, 2.5, 3.0]
    roi_list = []
    
    # for each period and multiplier, perform backtest
    for period, multiplier in [(x,y) for x in atr_period for y in atr_multiplier]:
        new_df = df
        supertrend = Supertrend(df, period, multiplier)
        new_df = df.join(supertrend)
        new_df = new_df[period:]
        entry, exit, roi = backtest_supertrend(new_df, 100000)
        roi_list.append((period, multiplier, roi))
    
    print(pd.DataFrame(roi_list, columns=['ATR_period','Multiplier','ROI']))
    
    # return the best parameter set
    return max(roi_list, key=lambda x:x[2])
df = yf.download('BTC-USD', start='2020-01-01')
optimal_param = find_optimal_parameter(df)
print(f'Best parameter set for BTC: ATR Period={optimal_param[0]}, Multiplier={optimal_param[1]}, ROI={optimal_param[2]}')

[*********************100%***********************]  1 of 1 completed
Buy 0 BTC at 7879.07 on 2020/01/09
Sell BTC at 8406.52 on 2020/01/23
Buy 0 BTC at 8909.82 on 2020/01/27
Sell BTC at 9889.42 on 2020/02/15
Buy 0 BTC at 10142.0 on 2020/02/18
Sell BTC at 9341.71 on 2020/02/25
Buy 0 BTC at 9078.76 on 2020/03/05
Sell BTC at 8108.12 on 2020/03/08
Buy 0 BTC at 6191.19 on 2020/03/19
Sell BTC at 5922.04 on 2020/03/29
Buy 0 BTC at 6606.78 on 2020/04/01
Sell BTC at 6865.49 on 2020/04/10
Buy 0 BTC at 7116.8 on 2020/04/16
Sell BTC at 6881.96 on 2020/04/20
Buy 0 BTC at 7429.72 on 2020/04/23
Sell BTC at 8756.43 on 2020/05/10
Buy 0 BTC at 9269.99 on 2020/05/13
Sell BTC at 9081.76 on 2020/05/21
Buy 0 BTC at 9525.75 on 2020/05/28
Sell BTC at 9321.78 on 2020/06/11
Buy 0 BTC at 9648.72 on 2020/06/22
Sell BTC at 9313.61 on 2020/06/24
Buy 0 BTC at 9375.47 on 2020/07/06
Sell BTC at 9132.23 on 2020/07/16
Buy 0 BTC at 9374.89 on 2020/07/21
Sell BTC at 11053.61 on 2020/08/02
Buy 0 BTC at 11747.02 on 2020/08/0

In [23]:
# Eth Price Data
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)
    
    
atr_period = 6
atr_multiplier = 2.5

symbol = 'ETH-USD'
df = yf.download(symbol, start='2020-01-01')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)
def backtest_supertrend(df, investment):
    is_uptrend = df['Supertrend']
    close = df['Close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = (equity / close[i] / 100) * 100
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} shares at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100k in Ethereum ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity
entry, exit, roi = backtest_supertrend(df, 100000)

# df.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close']),
                go.Scatter(x=df.index, y=df['Final Upperband'], line=dict(color='red', width=1), name="Final Upperband"),
                go.Scatter(x=df.index, y=df["Final Lowerband"], line=dict(color='green', width=1), name="Final Lowerband")])         
                

fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(title="Ethereum Price Data 2020 to Present", yaxis_title="Price in USD", autosize=False, width=1600, height=800,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
# update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
fig.update_layout(height=900, width=1600, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)])

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)

# Plot volume trace on 2nd row
colors = ['green' if row['Open'] - row['Close'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors
                    ), row=2, col=1)

# Assigning MACD
eth_df = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            )
# Plot MACD trace on 3rd row
fig.add_trace(go.Bar(x=df.index, 
                     y=eth_df.macd_diff()
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=eth_df.macd(),
                         line=dict(color='black', width=2)
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=eth_df.macd_signal(),
                         line=dict(color='blue', width=1)
                        ), row=3, col=1)
# Plot MACD trace on 3rd row
colors = ['green' if val >= 0 
          else 'red' for val in eth_df.macd_diff()]
fig.add_trace(go.Bar(x=df.index, 
                     y=eth_df.macd_diff(),
                     marker_color=colors
                    ), row=3, col=1)


[*********************100%***********************]  1 of 1 completed
Buy 700 shares at 134.17 on 2020/01/03
Sell at 225.68 on 2020/02/26
Buy 900 shares at 169.14 on 2020/04/06
Sell at 188.6 on 2020/05/10
Buy 800 shares at 219.84 on 2020/05/28
Sell at 222.96 on 2020/06/27
Buy 700 shares at 246.67 on 2020/07/08
Sell at 385.67 on 2020/09/03
Buy 700 shares at 370.97 on 2020/10/10
Sell at 1570.2 on 2021/02/23
Buy 600 shares at 1834.73 on 2021/03/08
Sell at 2460.68 on 2021/05/19
Buy 600 shares at 2321.72 on 2021/07/04
Sell at 1940.08 on 2021/07/13
Buy 500 shares at 2124.78 on 2021/07/23
Sell at 3426.39 on 2021/09/07
Buy 500 shares at 3518.52 on 2021/10/05
Sell at 4216.37 on 2021/11/16
Buy 700 shares at 2983.59 on 2022/02/04
Sell at 2628.65 on 2022/02/20
Buy 600 shares at 2945.34 on 2022/03/18
Sell at 2981.05 on 2022/04/11
Buy 1600 shares at 1233.13 on 2022/07/15
Earning from investing $100k in Ethereum $2738663.88 (ROI = 2738.66%)


In [24]:
# Eth Optimal Parameters
def find_optimal_parameter(df):
    # predefine several parameter sets
    atr_period = [6, 7, 8, 9, 10]
    atr_multiplier = [1.0, 1.5, 2.0, 2.5, 3.0]
    roi_list = []
    
    # for each period and multiplier, perform backtest
    for period, multiplier in [(x,y) for x in atr_period for y in atr_multiplier]:
        new_df = df
        supertrend = Supertrend(df, period, multiplier)
        new_df = df.join(supertrend)
        new_df = new_df[period:]
        entry, exit, roi = backtest_supertrend(new_df, 100000)
        roi_list.append((period, multiplier, roi))
    
    print(pd.DataFrame(roi_list, columns=['ATR_period','Multiplier','ROI']))
    
    # return the best parameter set
    return max(roi_list, key=lambda x:x[2])
df = yf.download('ETH-USD', start='2020-01-01')
optimal_param = find_optimal_parameter(df)
print(f'Best parameter set for ETH: ATR Period={optimal_param[0]}, Multiplier={optimal_param[1]}, ROI={optimal_param[2]}')

[*********************100%***********************]  1 of 1 completed
Buy 700 shares at 138.98 on 2020/01/09
Sell at 162.93 on 2020/01/23
Buy 600 shares at 170.93 on 2020/01/27
Sell at 257.95 on 2020/02/20
Buy 600 shares at 243.53 on 2020/03/06
Sell at 200.69 on 2020/03/08
Buy 1000 shares at 136.59 on 2020/03/19
Sell at 158.41 on 2020/04/10
Buy 900 shares at 172.16 on 2020/04/16
Sell at 172.3 on 2020/04/20
Buy 800 shares at 185.03 on 2020/04/23
Sell at 204.06 on 2020/05/06
Buy 900 shares at 199.19 on 2020/05/13
Sell at 199.88 on 2020/05/21
Buy 800 shares at 219.84 on 2020/05/28
Sell at 231.7 on 2020/06/11
Buy 700 shares at 242.53 on 2020/06/22
Sell at 232.94 on 2020/06/25
Buy 700 shares at 241.51 on 2020/07/06
Sell at 233.64 on 2020/07/16
Buy 700 shares at 245.02 on 2020/07/21
Sell at 406.46 on 2020/08/19
Buy 600 shares at 428.4 on 2020/08/30
Sell at 385.67 on 2020/09/03
Buy 600 shares at 387.18 on 2020/09/12
Sell at 341.79 on 2020/09/21
Buy 600 shares at 355.49 on 2020/09/26
Sell at 34

In [25]:
# Binance Price Data
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)
    
    
atr_period = 7
atr_multiplier = 2.0

symbol = 'BNB-USD'
df = yf.download(symbol, start='2020-01-01')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)
def backtest_supertrend(df, investment):
    is_uptrend = df['Supertrend']
    close = df['Close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = (equity / close[i] / 100) * 100
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} BNB at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100k into Binance is ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity
entry, exit, roi = backtest_supertrend(df, 100000)

# df.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close']),
                go.Scatter(x=df.index, y=df['Final Upperband'], line=dict(color='red', width=1), name="Final Upperband"),
                go.Scatter(x=df.index, y=df["Final Lowerband"], line=dict(color='green', width=1), name="Final Lowerband")])         
                

fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(title="Binance Price Data 2020 to Present", yaxis_title="Price in USD", autosize=False, width=1600, height=800,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])
# update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
fig.update_layout(height=900, width=1600, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(values=dt_breaks)])

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="MACD", showgrid=False, row=3, col=1)

# Plot volume trace on 2nd row
colors = ['green' if row['Open'] - row['Close'] >= 0 
          else 'red' for index, row in df.iterrows()]
fig.add_trace(go.Bar(x=df.index, 
                     y=df['Volume'],
                     marker_color=colors
                    ), row=2, col=1)

# Assigning MACD
btc_df = MACD(close=df['Close'], 
            window_slow=26,
            window_fast=12, 
            )
# Plot MACD trace on 3rd row
fig.add_trace(go.Bar(x=df.index, 
                     y=btc_df.macd_diff()
                    ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=btc_df.macd(),
                         line=dict(color='black', width=2)
                        ), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index,
                         y=btc_df.macd_signal(),
                         line=dict(color='blue', width=1)
                        ), row=3, col=1)
# Plot MACD trace on 3rd row
colors = ['green' if val >= 0 
          else 'red' for val in btc_df.macd_diff()]
fig.add_trace(go.Bar(x=df.index, 
                     y=btc_df.macd_diff(),
                     marker_color=colors
                    ), row=3, col=1)

[*********************100%***********************]  1 of 1 completed
Buy 7300 BNB at 13.66 on 2020/01/03
Sell at 22.33 on 2020/02/19
Buy 11800 BNB at 13.82 on 2020/04/04
Sell at 15.3 on 2020/05/10
Buy 10500 BNB at 17.15 on 2020/05/19
Sell at 16.31 on 2020/06/11
Buy 10500 BNB at 16.3 on 2020/07/06
Sell at 20.62 on 2020/09/03
Buy 8800 BNB at 24.53 on 2020/09/09
Sell at 23.27 on 2020/09/21
Buy 7100 BNB at 28.84 on 2020/09/29
Sell at 28.51 on 2020/10/30
Buy 6700 BNB at 30.39 on 2020/11/21
Sell at 28.2 on 2020/11/26
Buy 6200 BNB at 30.03 on 2020/12/14
Sell at 511.96 on 2021/05/17
Buy 10100 BNB at 313.6 on 2021/07/27
Sell at 417.51 on 2021/09/07
Buy 10000 BNB at 421.64 on 2021/10/01
Sell at 578.09 on 2021/11/17
Buy 11800 BNB at 487.01 on 2022/01/12
Sell at 440.01 on 2022/01/20
Buy 12600 BNB at 414.2 on 2022/02/05
Sell at 380.83 on 2022/02/20
Buy 11700 BNB at 408.48 on 2022/03/01
Sell at 361.85 on 2022/03/13
Buy 10500 BNB at 404.31 on 2022/03/22
Sell at 394.09 on 2022/04/11
Buy 12600 BNB at 3

In [26]:
# BNB Optimal Parameters
def find_optimal_parameter(df):
    # predefine several parameter sets
    atr_period = [6, 7, 8, 9, 10]
    atr_multiplier = [1.0, 1.5, 2.0, 2.5, 3.0]
    roi_list = []
    
    # for each period and multiplier, perform backtest
    for period, multiplier in [(x,y) for x in atr_period for y in atr_multiplier]:
        new_df = df
        supertrend = Supertrend(df, period, multiplier)
        new_df = df.join(supertrend)
        new_df = new_df[period:]
        entry, exit, roi = backtest_supertrend(new_df, 100000)
        roi_list.append((period, multiplier, roi))
    
    print(pd.DataFrame(roi_list, columns=['ATR_period','Multiplier','ROI']))
    
    # return the best parameter set
    return max(roi_list, key=lambda x:x[2])
df = yf.download('BNB-USD', start='2020-01-01')
optimal_param = find_optimal_parameter(df)
print(f'Best parameter set for BNB: ATR Period={optimal_param[0]}, Multiplier={optimal_param[1]}, ROI={optimal_param[2]}')

[*********************100%***********************]  1 of 1 completed
Buy 6800 BNB at 14.5 on 2020/01/09
Sell at 16.98 on 2020/01/25
Buy 6400 BNB at 18.12 on 2020/01/28
Sell at 24.15 on 2020/02/15
Buy 7400 BNB at 20.8 on 2020/03/05
Sell at 16.97 on 2020/03/08
Buy 10300 BNB at 12.28 on 2020/03/19
Sell at 11.44 on 2020/03/29
Buy 9000 BNB at 13.07 on 2020/04/02
Sell at 13.74 on 2020/04/10
Buy 8200 BNB at 15.03 on 2020/04/13
Sell at 15.09 on 2020/04/20
Buy 7600 BNB at 16.31 on 2020/04/25
Sell at 16.66 on 2020/05/06
Buy 7900 BNB at 16.14 on 2020/05/13
Sell at 16.12 on 2020/05/21
Buy 7400 BNB at 17.06 on 2020/05/28
Sell at 16.31 on 2020/06/11
Buy 7300 BNB at 16.48 on 2020/06/22
Sell at 15.92 on 2020/06/25
Buy 7400 BNB at 15.83 on 2020/07/01
Sell at 17.14 on 2020/07/16
Buy 7100 BNB at 17.96 on 2020/07/19
Sell at 21.26 on 2020/08/11
Buy 6500 BNB at 23.07 on 2020/08/14
Sell at 22.31 on 2020/08/19
Buy 6300 BNB at 23.08 on 2020/08/27
Sell at 20.62 on 2020/09/03
Buy 5600 BNB at 23.19 on 2020/09/06


In [27]:
# BTC Backtest
symbol = 'BTC-USD'
df = yf.download(symbol, start='2020-01-01')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)
def backtest_supertrend(df, investment):
    is_uptrend = df['Supertrend']
    close = df['Close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = (equity / close[i] / 100) * 100
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} BTC at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell BTC at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100k in Bitcoin is ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity
entry, exit, roi = backtest_supertrend(df, 100000)

[*********************100%***********************]  1 of 1 completed
Buy 13.614918387273029 BTC at 7344.88 on 2020/01/03
Sell BTC at 9690.14 on 2020/02/17
Buy 19.588618469750028 BTC at 6734.8 on 2020/03/24
Sell BTC at 8756.43 on 2020/05/10
Buy 17.681861653536007 BTC at 9700.41 on 2020/05/30
Sell BTC at 9264.81 on 2020/06/25
Buy 17.19768005707988 BTC at 9525.36 on 2020/07/22
Sell BTC at 10245.3 on 2020/09/03
Buy 15.881091769796141 BTC at 11094.35 on 2020/09/19
Sell BTC at 10246.19 on 2020/09/23
Buy 14.706154459103878 BTC at 11064.46 on 2020/10/09
Sell BTC at 17150.62 on 2020/11/26
Buy 12.85115764752176 BTC at 19625.84 on 2020/11/30
Sell BTC at 33922.96 on 2021/01/12
Buy 11.633840564973095 BTC at 37472.09 on 2021/02/03
Sell BTC at 48824.43 on 2021/02/23
Buy 10.360597021066704 BTC at 54824.12 on 2021/03/09
Sell BTC at 52774.27 on 2021/03/24
Buy 9.25222807224918 BTC at 59095.81 on 2021/04/01
Sell BTC at 56216.18 on 2021/04/18
Buy 9.00637868066329 BTC at 57750.18 on 2021/04/30
Sell BTC at 4

In [28]:
# ETH Backtest
symbol = 'ETH-USD'
df = yf.download(symbol, start='2020-01-01')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)
def backtest_supertrend(df, investment):
    is_uptrend = df['Supertrend']
    close = df['Close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = (equity / close[i] / 100) * 100
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} ETH at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell ETH at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100k in Ethereum is ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity
entry, exit, roi = backtest_supertrend(df, 100000)

[*********************100%***********************]  1 of 1 completed
Buy 745.3136143354595 ETH at 134.17 on 2020/01/03
Sell ETH at 225.68 on 2020/02/26
Buy 994.4523670057551 ETH at 169.14 on 2020/04/06
Sell ETH at 188.6 on 2020/05/10
Buy 874.24884937881 ETH at 214.53 on 2020/05/18
Sell ETH at 227.14 on 2020/06/19
Buy 813.3392301240542 ETH at 244.14 on 2020/06/23
Sell ETH at 222.96 on 2020/06/27
Buy 750.8458396366665 ETH at 241.51 on 2020/07/06
Sell ETH at 389.13 on 2020/08/21
Buy 682.0070344900129 ETH at 428.4 on 2020/08/30
Sell ETH at 385.67 on 2020/09/03
Buy 719.4551949440645 ETH at 365.59 on 2020/10/09
Sell ETH at 518.8 on 2020/11/26
Buy 607.0646757480415 ETH at 614.84 on 2020/11/30
Sell ETH at 583.71 on 2020/12/23
Buy 519.0822433660754 ETH at 682.64 on 2020/12/27
Sell ETH at 1570.2 on 2021/02/23
Buy 444.2402613894248 ETH at 1834.73 on 2021/03/08
Sell ETH at 1593.41 on 2021/03/24
Buy 388.99778432865565 ETH at 1819.68 on 2021/03/29
Sell ETH at 2166.19 on 2021/04/19
Buy 332.4694024414

In [29]:
# BNB Backtest
symbol = 'BNB-USD'
df = yf.download(symbol, start='2020-01-01')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)
def backtest_supertrend(df, investment):
    is_uptrend = df['Supertrend']
    close = df['Close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = (equity / close[i] / 100) * 100
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} BNB at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell BNB at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100k in Binance is ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity
entry, exit, roi = backtest_supertrend(df, 100000)

[*********************100%***********************]  1 of 1 completed
Buy 7300 BNB at 13.66 on 2020/01/03
Sell BNB at 22.33 on 2020/02/19
Buy 11800 BNB at 13.82 on 2020/04/04
Sell BNB at 15.3 on 2020/05/10
Buy 10500 BNB at 17.15 on 2020/05/19
Sell BNB at 16.31 on 2020/06/11
Buy 10500 BNB at 16.3 on 2020/07/06
Sell BNB at 20.62 on 2020/09/03
Buy 8800 BNB at 24.53 on 2020/09/09
Sell BNB at 23.27 on 2020/09/21
Buy 7100 BNB at 28.84 on 2020/09/29
Sell BNB at 28.51 on 2020/10/30
Buy 6700 BNB at 30.39 on 2020/11/21
Sell BNB at 28.2 on 2020/11/26
Buy 6200 BNB at 30.03 on 2020/12/14
Sell BNB at 511.96 on 2021/05/17
Buy 10100 BNB at 313.6 on 2021/07/27
Sell BNB at 417.51 on 2021/09/07
Buy 10000 BNB at 421.64 on 2021/10/01
Sell BNB at 578.09 on 2021/11/17
Buy 11800 BNB at 487.01 on 2022/01/12
Sell BNB at 440.01 on 2022/01/20
Buy 12600 BNB at 414.2 on 2022/02/05
Sell BNB at 380.83 on 2022/02/20
Buy 11700 BNB at 408.48 on 2022/03/01
Sell BNB at 361.85 on 2022/03/13
Buy 10500 BNB at 404.31 on 2022/0

In [33]:
# TFUEL Backtest
symbol = 'TFUEL-USD'
df = yf.download(symbol, start='2020-01-01')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)
def backtest_supertrend(df, investment):
    is_uptrend = df['Supertrend']
    close = df['Close']
    
    # initial condition
    in_position = False
    equity = investment
    commission = 5
    share = 0
    entry = []
    exit = []
    
    for i in range(2, len(df)):
        # if not in position & price is on uptrend -> buy
        if not in_position and is_uptrend[i]:
            share = (equity / close[i] / 100) * 100
            equity -= share * close[i]
            entry.append((i, close[i]))
            in_position = True
            print(f'Buy {share} TFUEL at {round(close[i],4)} on {df.index[i].strftime("%Y/%m/%d")}')
        # if in position & price is not on uptrend -> sell
        elif in_position and not is_uptrend[i]:
            equity += share * close[i] - commission
            exit.append((i, close[i]))
            in_position = False
            print(f'Sell TFUEL at {round(close[i],4)} on {df.index[i].strftime("%Y/%m/%d")}')
    # if still in position -> sell all share 
    if in_position:
        equity += share * close[i] - commission
    
    earning = equity - investment
    roi = round(earning/investment*100,2)
    print(f'Earning from investing $100k in TFUEL is ${round(earning,2)} (ROI = {roi}%)')
    return entry, exit, equity
entry, exit, roi = backtest_supertrend(df, 100000)

[*********************100%***********************]  1 of 1 completed
Buy 41928721.17793367 TFUEL at 0.0024 on 2020/01/03
Sell TFUEL at 0.0026 on 2020/02/26
Buy 56854324.207105905 TFUEL at 0.0019 on 2020/04/29
Sell TFUEL at 0.0085 on 2020/09/03
Buy 42557736.350356884 TFUEL at 0.0114 on 2020/09/29
Sell TFUEL at 0.0085 on 2020/11/04
Buy 34628993.527485125 TFUEL at 0.0104 on 2020/11/24
Sell TFUEL at 0.2511 on 2021/04/22
Buy 20168725.263860848 TFUEL at 0.4311 on 2021/06/03
Sell TFUEL at 0.353 on 2021/07/05
Buy 20005080.99277889 TFUEL at 0.3558 on 2021/07/25
Sell TFUEL at 0.3076 on 2021/09/10
Buy 18609179.61371231 TFUEL at 0.3307 on 2021/10/26
Sell TFUEL at 0.3122 on 2021/11/16
Buy 28881099.529362824 TFUEL at 0.2011 on 2022/02/05
Sell TFUEL at 0.165 on 2022/02/21
Buy 24501007.691772703 TFUEL at 0.1945 on 2022/03/27
Sell TFUEL at 0.1592 on 2022/04/11
Buy 63470304.61967956 TFUEL at 0.0614 on 2022/07/17
Earning from investing $100k in TFUEL is $4024676.09 (ROI = 4024.68%)
